![logo](../../_static/images/NCI_logo.png)

-------

# Use cdo to create land/sea mask


In this notebook:

- Load CDO module
- Look at the information of the file
- Visualise the data using ncview 
- Create land and ocean masks to limit the field to land/ocean values 

The following material uses Coupled Model Intercomparison Project (CMIP6) collections. The CMIP6 terms of use are found [here](https://pcmdi.llnl.gov/CMIP6/TermsOfUse/TermsOfUse6-1.html). For more information on the collection, please [click here](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f6600_2266_8675_3563).


CDO stands for "Climate Data Operators". CDO is a collection of command line Operators to manipulate and analyse Climate and NWP model Data.
Supported data formats are GRIB 1/2, netCDF 3/4, SERVICE, EXTRA and IEG. There are more than 600 operators available. See [here](https://code.mpimet.mpg.de/projects/cdo) for more information about this library.

### Load CDO module 

```
$ module load cdo
```

### List the path of the CMIP6 data

Here we will be using CMIP6 data as an example. 

```
$ ls /g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc
```
This file includes the near surface temperature from the 20th century all-forcing historical simulation based on the CESM2.0 model of NCAR. 

### Have a look at the data file using cdo info

Basic usage: **cdo info filename | less**
```
$ cdo info /g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc | less
```

**less** display only one page at a time in the terminal. You can move forwards and backwards to see more. Press **q** to quit the view.

![1](images/cdo_anomoly_nino1.png)

Use **ncview** to show the data:
```
$ ncview /g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc
```

The data looks like this 

![1](images/cdo_anomoly_nino3.png)

### Let's see which year this file includes

We use function **showyear** to display all the years in this file

**Basic usage**: **cdo showyear file**
```
$ cdo showyear /g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc
```

![2](images/cdo_anomoly_nino2.png)

### Create topography data 

```
$ cdo -f nc topo topo.nc
```
The operator topo creates a half degree regular grid with the heights and depths. This can be very useful for generating masks to a certain height/depth.

The created topo.nc file is shown below. Here in the topo.nc file, the topography values over land areas are greater than 0 whereas over ocean areas the topography is less than 0.


![3](images/cdo_mask1.png)

### Create ocean mask file

**Basic usage**: **cdo gtc, 0 -remapcon, input.nc topo.nc output.nc**


There are several steps involved in here:

First, remapping topo.nc file to change its resolution so that it is consistent with the input data that we use: **cdo remapcon, input.nc topo.nc output.nc**

```
$ cdo -remapcon,/g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc topo.nc topo_adjusted.nc
```

Second, create a mask containing 1 if the topography element is greater than 0: **cdo gtc, 0 input.nc output.nc**
```
$ cdo gtc, 0 topo_adjusted.nc seamask.nc
```

### Now we will try to chain all the commands together

**Basic Usage: cdo -f nc -gtc, 0 -remapcon, input.nc -topo output.nc**
```
$ cdo -f nc  -gtc,0  -remapcon,/g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc -topo seamask.nc
```

### Similarly, we can get land mask using the following commands

```
cdo -f nc  -lec, 0  -remapcon,/g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc -topo landmask.nc
```

### Let's have a look at what the generated seamask.nc looks like

```
$ module load ncview
$ ncview seamask.nc
```

![4](images/cdo_mask2.png)

Now we get a topography file where all ocean values are 0 and land values are 1.

### Now we can apply the seamask to the original data to mask out ocean data

Multiply input.nc and seamask, and the ocean values in output.nc become all 0.

**cdo mul input.nc seamask.nc output.nc**

Set 0 as missing value in the data:

**cdo setmissval, 0 input.nc output.nc**


To combine these two commands together:

**cdo setmissval, 0 -mul input.nc seamask.nc output.nc**
``` 
$ cdo setmissval, 0 -mul /g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc seamask.nc tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412_land.nc
```

Warning: Be cautious when using 0 as the missing value in case that 0 has already been used as a reasonable element in the original data. In this example though, the data use Kelvin(K) as the unit of temperature, therefore using 0 as the missing value is not an issue because 0 cannot be a reasonable earth near surface temperature.

### Let's have a look at the new land data file

```
$ ncview tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412_land.nc
```

![5](images/cdo_mask3.png)